In [1]:
import time

from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from awkward import JaggedArray
import uproot_methods
from uproot_methods import *
import numpy as np
import glob as glob
import itertools
import json
import uproot_methods
import copy

In [2]:
# Look at ProcessorABC to see the expected methods and what they are supposed to do
class JetMassProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")

        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", np.array([200,260,350,460,550,650,760]))
        m_axis = hist.Bin("m", r"Mass [GeV]", np.array([0,5,10,20,40,60,80,100,150,200,250,300,350]))
        ptreco_axis = hist.Bin("ptreco", r"$p_{T}$ [GeV]", np.array([200,260,350,460,550,650,760]))
        mreco_axis = hist.Bin("mreco", r"Mass [GeV]", np.array([0,5,10,20,40,60,80,100,150,200,250,300,350]))
        ptgen_axis = hist.Bin("ptgen", r"$p_{T}$ [GeV]", np.array([200,260,350,460,550,650,760]))
        mgen_axis = hist.Bin("mgen", r"Mass [GeV]", np.array([0,2.5,5,7.5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,325,350]))


        r_axis = hist.Bin("r", "RECO / GEN response", 100, 0, 2)
        dr_axis = hist.Bin("dr", r"$\Delta r$", 80, 0, 0.8)
        
        self._accumulator = processor.dict_accumulator({
            'pt':hist.Hist("Counts", dataset_axis, pt_axis),
            'm':hist.Hist("Counts", dataset_axis, m_axis),
            'msd':hist.Hist("Counts", dataset_axis, m_axis),
            'pt_v_m':hist.Hist("Counts", dataset_axis, pt_axis, m_axis ),
            'pt_v_msd':hist.Hist("Counts", dataset_axis, pt_axis, m_axis ),
            'dr':hist.Hist("Counts", dataset_axis, dr_axis),
            'r_pt_ptvm':hist.Hist("Counts", dataset_axis, pt_axis, m_axis, r_axis),
            'r_m_ptvm':hist.Hist("Counts", dataset_axis, pt_axis, m_axis, r_axis),
            'r_msd_ptvmsd':hist.Hist("Counts", dataset_axis, pt_axis, m_axis, r_axis),
            'response_m':hist.Hist("Counts", dataset_axis, ptgen_axis, mgen_axis, ptreco_axis, mreco_axis),
            'response_msd':hist.Hist("Counts", dataset_axis, ptgen_axis, mgen_axis, ptreco_axis, mreco_axis),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def makeCuts(self, cut, a):
        for i in a:
            i=i[cut]
    
    def process(self, df):
        output = self.accumulator.identity()

        #print("1")
        dataset = df['dataset']
        muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'],
            eta=df['Muon_eta'],
            phi=df['Muon_phi'],
            mass=df['Muon_mass'],
            charge=df['Muon_charge'],
            softId=df['Muon_softId'],
            tightId=df['Muon_tightId']
            )
        Jets = JaggedCandidateArray.candidatesfromcounts(
            df['nFatJet'],
            pt=df['FatJet_pt'],
            eta=df['FatJet_eta'],
            phi=df['FatJet_phi'],
            mass=df['FatJet_mass'],
            msoftdrop=np.where( df['FatJet_msoftdrop'] >= 0,df['FatJet_msoftdrop'],-1),
            jetId=df['FatJet_jetId']
            )
        GenJets = JaggedCandidateArray.candidatesfromcounts(
            df['nGenJetAK8'],
            pt=df['GenJetAK8_pt'],
            eta=df['GenJetAK8_eta'],
            phi=df['GenJetAK8_phi'],
            mass=df['GenJetAK8_mass']
            )
        GenSubJets = JaggedCandidateArray.candidatesfromcounts(
            df['nSubGenJetAK8'],
            pt=df['SubGenJetAK8_pt'],
            eta=df['SubGenJetAK8_eta'],
            phi=df['SubGenJetAK8_phi'],
            mass=df['SubGenJetAK8_mass']
            )
        


        # Require at least one reco jet that passes jet id
        output['cutflow']['all events'] += Jets.size
         
        jetId_cut = (Jets.jetId > 0)
        Jets = Jets[jetId_cut]
        oneJet = (Jets.counts >=1)
        self.makeCuts( oneJet, [Jets,GenJets,GenSubJets,muons])
        output['cutflow']['>=1 with loose id'] += oneJet.sum() 

        
        # Select dimuon events
        soft_id = (muons.softId > 0)
        muons = muons[soft_id]
        twoMuons = (muons.counts >= 2)
        self.makeCuts( twoMuons, [Jets,GenJets,GenSubJets,muons])
        output['cutflow']['>=2 soft muons'] += twoMuons.sum()


        
        dimuons = muons.distincts()
        opposite_charge = (dimuons.i0['charge'] * dimuons.i1['charge'] == -1)          
        zmasscut = ((dimuons.mass > 50) & (dimuons.mass < 110))
        dimuons = dimuons[opposite_charge & zmasscut]        
        oneZcand = (dimuons.counts >= 1)   
        self.makeCuts(oneZcand, [Jets,GenJets,GenSubJets])

        output['cutflow']['Z selection'] += oneZcand.sum()        

        Zcands = dimuons.i0['p4'] + dimuons.i1['p4']
        
        zjetmatch = Zcands.cross( Jets, nested=True )        
        zjetcutmetric = zjetmatch.i0.delta_phi(zjetmatch.i1.p4)
        zjetmatch = zjetmatch[(zjetcutmetric > np.pi * 0.5) & (zjetcutmetric < np.pi * 1.5) ]
        leadjetmetric = zjetmatch.i1.p4.pt
        leadjetindex = leadjetmetric.argmax()
        leadjet = zjetmatch[leadjetindex].i1
        #leadjet = zjetsortmetric.argmin()
        
        #dphi_cut = (zjetcutmetric[leadjet] > np.pi * 0.5)
        #zjetmatch = zjetmatch[dphi_cut]
        #self.makeCuts(dphi_cut, [zjetmatch])
        #output['cutflow']['dphi'] += dphi_cut.sum()  

        #dphi_cut = Zcands[:,0].delta_phi( Jets.p4[:,0] ) > np.pi * 0.5
        #self.makeCuts( dphi_cut.any(), [Zcands,Jets,GenJets,GenSubJets])
        #output['cutflow']['dPhi(Z,jet) cut'] += dphi_cut.any().sum()
        
        
        # Match gen <---> gen subjets
        
        gensubpairs = GenJets.cross( GenSubJets, nested=True )
        gensubjetmetric = gensubpairs.i0.p4.delta_r(gensubpairs.i1.p4)
        dr_cut = (gensubjetmetric < 0.8)
        gensubpairs = gensubpairs[dr_cut]
        gensubjets = gensubpairs.i1        
        gengroomed = gensubjets.p4.sum()
        # Add the groomed p4 and mass to the GenJet table
        GenJets.add_attributes( sdp4=gengroomed )
        GenJets.add_attributes( msoftdrop=gengroomed.mass )
        


                
        # Match reco <---> gen
        recogenpairs = Jets.cross(GenJets, nested=True)
        metric = recogenpairs.i0.p4.delta_r( recogenpairs.i1.p4 )
        index_of_minimized = metric.argmin()
        dr_cut2 = (metric[index_of_minimized] < 0.4)
        recogenpairs = recogenpairs[index_of_minimized][dr_cut2]
        kinreco = (recogenpairs.i0.pt > 200) & (np.abs(recogenpairs.i0.eta) < 2.5)
        kingen = (recogenpairs.i1.pt > 200) & (np.abs(recogenpairs.i1.eta) < 2.5)
        recogenpairs = recogenpairs[ (kinreco) & (kingen)]
        
        #badcase1 = (recogenpairs.i0.msoftdrop > 0) & (recogenpairs.i0.msoftdrop < 10.) & (recogenpairs.i1.msoftdrop > 30.)
        #badcase2 = (recogenpairs.i0.msoftdrop > 30.) & (recogenpairs.i1.msoftdrop < 10.)
        #badpairs = recogenpairs[badcase1 | badcase2]
        #badpairs = recogenpairs[badcase1]
        #print("bad pairs:")
        #print(badpairs.i0.msoftdrop[badpairs.counts > 0].flatten().flatten()[:10])
        #print(badpairs.i1.msoftdrop[badpairs.counts > 0].flatten().flatten()[:10])
        #rbad = badpairs.i0.msoftdrop[badpairs.counts > 0] / badpairs.i1.msoftdrop[badpairs.counts > 0]
        #print(rbad.flatten().flatten()[:10])
        
                
        # Make some plots)
        ptreco = recogenpairs.i0.pt.flatten().flatten()
        ptgen = recogenpairs.i1.pt.flatten().flatten()         
        mreco = recogenpairs.i0.mass.flatten().flatten()
        msdreco = recogenpairs.i0.msoftdrop.flatten().flatten()
        mgen = recogenpairs.i1.mass.flatten().flatten()
        msdgen = recogenpairs.i1.msoftdrop.flatten().flatten()
        #print("ptreco:", ptreco)
        #print("ptgen:", ptgen)
        #print("mreco:", mreco)
        #print("mgen:", mgen)
        #print("msdreco:", msdreco)
        #print("msdgen:", msdgen)

        ptreco_over_ptgen = np.where( ptgen > 0, ptreco/ptgen, -1)
        mreco_over_mgen = np.where( mgen > 0, mreco/mgen, -1)
        msdreco_over_msdgen = np.where( msdgen > 0, msdreco/msdgen, -1)
        

        
        output['pt'].fill(dataset=dataset,pt=ptreco)
        output['m'].fill(dataset=dataset,m=mreco)
        output['msd'].fill(dataset=dataset,m=msdreco)
        output['pt_v_m'].fill(dataset=dataset,pt=ptreco,m=mreco)
        output['pt_v_msd'].fill(dataset=dataset,pt=ptreco,m=msdreco)
        mcut = mgen>1.0
        msdcut = msdgen>1.0
        output['r_pt_ptvm'].fill(dataset=dataset,pt=ptgen[mcut],m=mgen[mcut],r=ptreco_over_ptgen[mcut])
        output['r_m_ptvm'].fill(dataset=dataset,pt=ptgen[mcut],m=mgen[mcut],r=mreco_over_mgen[mcut])
        output['r_msd_ptvmsd'].fill(dataset=dataset,pt=ptgen[msdcut],m=msdgen[msdcut],r=msdreco_over_msdgen[msdcut])
        output['response_m'].fill(dataset=dataset,ptgen=ptgen[mcut],mgen=mgen[mcut],ptreco=ptreco[mcut],mreco=mreco[mcut])
        output['response_msd'].fill(dataset=dataset,ptgen=ptgen[msdcut],mgen=msdgen[msdcut],ptreco=ptreco[msdcut],mreco=msdreco[msdcut])
        
        
        
        return output
        


    def postprocess(self, accumulator):
        return accumulator


In [3]:
# Can grab a file on cmslpc from 
# /store/group/lpctlbsm/NanoAODJMAR_2019_V1/Production/CRAB/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/DYJetsToLLM-50TuneCUETP8M113TeV-madgraphMLM-pythia8RunIISummer16MiniAODv3-PUMoriond17_ext2-v2/190513_171710/0000/*.root

#infiles = glob.glob('/mnt/data/cms/store/group/lpctlbsm/NanoAODJMAR_2019_V1/Production/CRAB/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/DYJetsToLLM-50TuneCUETP8M113TeV-madgraphMLM-pythia8RunIISummer16MiniAODv3-PUMoriond17_ext2-v2/190513_171710/0000/*.root')
infiles = glob.glob('/mnt/data/cms/store/group/lpctlbsm/NanoAODJMAR_2019_V1/Production/CRAB/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8/DYJetsToLLM-50TuneCUETP8M113TeV-madgraphMLM-pythia8RunIISummer16MiniAODv3-PUMoriond17_ext2-v2/190513_171710/*.root')

fileset = {"DY":infiles}

tstart = time.time() 
output = processor.run_uproot_job(fileset,
                                  treename='Events',
                                  processor_instance=JetMassProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers':4, 'flatten': True},
                                  chunksize=500000#, maxchunks=10
                                 )


elapsed = time.time() - tstart
print(output)

/usr/local/jupyter/lib/python3.6/site-packages/uproot_methods/classes/TLorentzVector.py:189: RuntimeWarning: invalid value encountered in sqrt
  return self._trymemo("mass", lambda self: self.awkward.numpy.sqrt(self.mag2))
/usr/local/jupyter/lib/python3.6/site-packages/uproot_methods/classes/TLorentzVector.py:189: RuntimeWarning: invalid value encountered in sqrt
  return self._trymemo("mass", lambda self: self.awkward.numpy.sqrt(self.mag2))
/usr/local/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:181: RuntimeWarning: invalid value encountered in greater
/usr/local/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:181: RuntimeWarning: invalid value encountered in greater
/usr/local/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:181: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:181: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/jupyter/lib/python3.6/sit

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/concurrent/futures/process.py", line 295, in _queue_management_worker
    shutdown_worker()
  File "/usr/lib64/python3.6/concurrent/futures/process.py", line 253, in shutdown_worker
    call_queue.put_nowait(None)
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 129, in put_nowait
    return self.put(obj, False)
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 83, in put
    raise Full
queue.Full



BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
for i,j in output['cutflow'].items():
    print( '%30s : %8.0f' % (i,j) )

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
response_m = output['response_m']
response_msd = output['response_msd']
nptgen = response_m.axis('ptgen').size - 2
nmgen = response_m.axis('mgen').size - 2
nptreco = response_m.axis('ptreco').size - 2
nmreco = response_m.axis('mreco').size - 2

print(nptgen, nmgen, nptreco, nmreco)


In [ ]:
response_m_2dproj = response_m.values()[('DY',)].reshape((nptgen-1)*(nmgen-1),(nptreco-1)*(nmreco-1))
response_msd_2dproj = response_msd.values()[('DY',)].reshape((nptgen-1)*(nmgen-1),(nptreco-1)*(nmreco-1))



In [ ]:
# Set fonts (from https://stackoverflow.com/questions/3899980/how-to-change-the-font-size-on-a-matplotlib-plot)
SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 150

In [ ]:
plt.imshow( np.log(response_m_2dproj) )

In [ ]:
plt.imshow( np.log(response_msd_2dproj) )

In [ ]:
fig, ax = hist.plotgrid(output['pt'], overlay="dataset", stack=False, density=True
                                 )
plt.yscale("log")
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()

In [ ]:
fig, ax = hist.plotgrid(output['m'], overlay="dataset", stack=False, density=True
                                 )
plt.yscale("log")
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def gauss(x, A, mu, sigma):    
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

In [ ]:
jmr = []
jms = []
mvals=[]

for i in output['r_msd_ptvmsd'].axis('pt')[1:-2]:
    for j in output['r_msd_ptvmsd'].axis('m')[1:-2] :
        #print(i,j)
        mvals.append( [i.lo,j.lo] )
        
        xvals = np.array( [k.lo for k in output['r_msd_ptvmsd'].axis('r')[1:-2]] )
        yvals = output['r_msd_ptvmsd'].integrate("pt", int_range=slice(i.lo,i.hi)).integrate("m", int_range=slice(j.lo,j.hi)).values()[('DY',)]
        if yvals.sum() < 100:
            jms.append( [0,0 ] )
            jmr.append( [0,0] )
            continue
        fig,ax,_ = hist.plot1d(output['r_msd_ptvmsd'].integrate("pt", int_range=slice(i.lo,i.hi)).integrate("m", int_range=slice(j.lo,j.hi)), 
                               overlay='dataset')
        plt.text(0.2,0.5, 'pt=%.0f-%.0f GeV m=%.0f-%.0f GeV' %(i.lo,i.hi,j.lo,j.hi), transform=ax.transAxes)
        plt.text(0.05,1.01, "CMS Preliminary", transform=ax.transAxes, fontsize=20)
        xavg = np.average(xvals,weights=yvals)
        xstd = np.sqrt( np.average((xvals-xavg)**2, weights=yvals))
        x1 = max(0.,xavg-2*xstd)
        x2 = min(2.,xavg+2*xstd)
        k1 = np.digitize(x1, xvals)
        k2 = np.digitize(x2, xvals)
        p,pcov = curve_fit(gauss, xvals[k1:k2],yvals[k1:k2], p0 = [1., 0., 1.])
        plt.plot( xvals, gauss(xvals,p[0],p[1],p[2]) )
        plt.text(0.2,0.4, r"$\mu=%6.2e$ $\sigma=%6.2e$"%(p[1],p[2]), transform=ax.transAxes)
        
        jms.append( [p[1],np.sqrt(pcov[1,1]) ] )
        jmr.append( [p[2],np.sqrt(pcov[2,2]) ] )

In [ ]:
jmr = np.array(jmr)
jms = np.array(jms)
mvals=np.array(mvals)